In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib import data_processors, tasks
from lib.bert import BertForSequenceClassification
from lib.train_eval import train, evaluate, predict

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
%env CUDA_VISIBLE_DEVICES=0

params = {
    'data_dir': '../datasets/CoLA',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'cola',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 1,
    'seed': 1331,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])

env: CUDA_VISIBLE_DEVICES=1


In [3]:
params['num_labels'] = tasks.num_labels[params['task_name']]
params['label_list'] = tasks.label_lists[params['task_name']]

processor = tasks.processors[params['task_name']]()
tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
dev_examples = processor.get_dev_examples(params['data_dir'])

model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

## Training and evaluation

In [4]:
TOTAL_EPOCH_NUMS = 5

for epoch_num in range(1, TOTAL_EPOCH_NUMS + 1):
    params['num_train_epochs'] = 1
    checkpoint_files = {
        'config': 'bert_config.json',
        'file_to_save': 'model_{}_epoch_{}.pth'.format(
            params['task_name'], epoch_num)
    }

    model, result = train(model, tokenizer, params,
                          train_examples,
                          valid_examples=dev_examples,
                          checkpoint_files=checkpoint_files)

Iteration:   0%|          | 0/268 [00:00<?, ?it/s]

***** Running training *****
Num examples: 8551
Batch size:   32
Num steps:    267

Epoch: 1


Evaluating:   0%|          | 0/131 [00:00<?, ?it/s]

{'train_loss': 0.49238246303996935, 'train_global_step': 268}
***** Running evaluation *****
Num examples:  1043
Batch size:    8


Evaluating: 100%|██████████| 131/131 [00:05<00:00, 26.15it/s]


...

***** Running training *****
Num examples: 8551
Batch size:   32
Num steps:    267

Epoch: 1


Iteration: 100%|██████████| 268/268 [02:01<00:00,  2.69it/s]


{'train_loss': 0.10236947970993038, 'train_global_step': 268}
***** Running evaluation *****
Num examples:  1043
Batch size:    8


Evaluating: 100%|██████████| 131/131 [00:05<00:00, 25.93it/s]


{'eval_loss': 0.7175850086579365, 'eval_accuracy': 0.8120805369127517, 'eval_f1_score': 0.8722294654498045, 'eval_matthews_corrcoef': 0.5355944718642581}


In [6]:
best_model = BertForSequenceClassification.from_pretrained(
    params['bert_model'],
    cache_dir=params['cache_dir'],
    num_labels=params['num_labels']).to(params['device'])

best_model.load_state_dict(torch.load(
    os.path.join(params['output_dir'], 'model_{}_epoch_{}.pth'.format(
        params['task_name'], 4))))

In [8]:
result, prob_preds = evaluate(best_model, tokenizer, params,
                              dev_examples)
result

***** Running evaluation *****
Num examples:  1043
Batch size:    8


Evaluating: 100%|██████████| 131/131 [00:04<00:00, 26.28it/s]


{'eval_loss': 0.6017286172740506,
 'eval_accuracy': 0.8216682646212847,
 'eval_f1_score': 0.8784313725490197,
 'eval_matthews_corrcoef': 0.5609903802347734}